In [3]:
from pathlib import Path
import pandas as pd
import csv
import sys

script_path = Path.cwd().parent / "scripts"
sys.path.insert(0, script_path)
print(sys.path)

from utils import *

data_path = Path.cwd().parent / "data"

[WindowsPath('C:/opt/justice40-tool/score/scripts'), WindowsPath('C:/opt/justice40-tool/score/scripts'), WindowsPath('C:/opt/justice40-tool/score/scripts'), 'C:\\opt\\justice40-tool\\score\\ipython', 'C:\\Python39\\python39.zip', 'C:\\Python39\\DLLs', 'C:\\Python39\\lib', 'C:\\Python39', 'c:\\opt\\justice40-tool\\score\\venv', '', 'c:\\opt\\justice40-tool\\score\\venv\\lib\\site-packages', 'c:\\opt\\justice40-tool\\score\\venv\\lib\\site-packages\\win32', 'c:\\opt\\justice40-tool\\score\\venv\\lib\\site-packages\\win32\\lib', 'c:\\opt\\justice40-tool\\score\\venv\\lib\\site-packages\\Pythonwin', 'c:\\opt\\justice40-tool\\score\\venv\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\j\\.ipython']


ModuleNotFoundError: No module named 'utils'

In [2]:
# store all fips codes in list
fips_state_list = get_state_fips_codes
fips_state_list

NameError: name 'get_state_fips_codes' is not defined

In [6]:
# EJSCreen ETL Load
csv_path = data_path / "dataset" / "ejscreen_2020"